# Example JSON Schema

In [1]:
import pathlib
import json_ntv
from json_ntv.namespace import from_file
from json_ntv.ntv_schema import ntv_validate, ntv_validate2
from json_ntv import Ntv
from jsonschema import validate

# Example 1

In [2]:
# example - overview - JSON Schema
data_product = {
    "name": "Widget",
    "price": 10.99,
    "quantity": 5 }

json_schema = {
    "title": "Product",
    "properties": {          
        "name": {
            "type": "string" },
        "price": {
            "type": "number", 
            "minimum": 0 },
        "quantity": {
            "type": "integer", 
            "minimum": 1 } },
    "required": [ "name", "price", "quantity"] }

try:
    validate(data_product, json_schema)
    print("Valid JSON object.")
except Exception as e:
    print("Invalid JSON object:", e)

Valid JSON object.


In [15]:
# example NTV Schema
print('NTV existing schema - valid :', ntv_validate(data_product, json_schema))

# example NTV Schema option 1
schema_opt1 = {
    "title": "Product",
    "/name": {
        "type": "string" },
    "/price": {
        "type": "number", 
        "minimum": 0 },
    "/quantity": {
        "type": "integer", 
        "minimum": 1 } ,
    "required": [ "/name", "/price", "/quantity"] }
print('NTV schema 1 - valid :', ntv_validate(data_product, schema_opt1))

# example NTV Schema option 2
schema = {"sch.": {
    ":title": "Product - NTVschema",
    "name": {
        ":type": "string" },
    "price": {
        ":type": "number", 
        ":minimum": 0 },
    "quantity": {
        ":type": "integer", 
        ":minimum": 1 },
    ":required": [ "name", "price", "quantity"] }}
print('NTV schema 2 - valid :', ntv_validate2(data_product, schema))

NTV existing schema - valid : True
NTV schema 1 - valid : True
NTV schema 2 - valid : True


## Example 2

In [5]:
json_test = {'family' : 'white', 
             'childrens': [24, 
                           15, 
                           {'walter jr': None}, 
                           {'judith': 21}]}

json_schema = {'properties': {'family':{
                                    'maxLength':10,
                                    'type': ['string', 'integer']},
                              'childrens':{
                                    'maxItems': 4,
                                    'items': {
                                        'maximum': 25, 
                                        'propertyNames': {'maxLength':10},
                                        'properties':{
                                            'judith': {'maximum': 23}
                                        }}}}}


validate(json_test, json_schema)
ntv_validate(json_test, json_schema, mode=2)

     /family - valueNtv  :  white {'maxLength': 10, 'type': ['string', 'integer']}
     /childrens/0 - valueNtv  :  24 {'maximum': 25}
     /childrens/0 - nameNtv  :   {'maxLength': 10}
     /childrens/1 - valueNtv  :  15 {'maximum': 25}
     /childrens/1 - nameNtv  :   {'maxLength': 10}
     /childrens/2 - valueNtv  :  None {'maximum': 25}
     /childrens/2 - nameNtv  :  walter jr {'maxLength': 10}
     /childrens/3 - valueNtv  :  21 {'maximum': 23}
     /childrens/3 - valueNtv  :  21 {'maximum': 25}
     /childrens/3 - nameNtv  :  judith {'maxLength': 10}
     /childrens - valueNtv  :  [24, 15, {"walter jr": null}, {"judith": 21}] {'maxItems': 4}


True

In [20]:
json_schema2 = {'properties': {'family':{
                                    'maxLength':10},
                              'childrens':{
                                    'maxItems': 4,
                                    'items': {
                                        'maximum': 25, 
                                        'propertyNames': {'maxLength':10}},
                                    '/1': {'minimum':16},
                                    '/judith': {'maximum': 20}
                                    }}}


validate(json_test, json_schema2)
ntv_validate(json_test, json_schema2, mode=1)

  validate :  
  validate :  /family
  validate :  /childrens
  validate :  /childrens/0
  validate :  /childrens/1
  validate :  /childrens/2
  validate :  /childrens/3
  validate :  /childrens/1
  error  /childrens/1 - valueNtv  :  15 is not valid with schema :  {'minimum': 16}
  validate :  /childrens/3
  error  /childrens/3 - valueNtv  :  21 is not valid with schema :  {'maximum': 20}


False

`properties` and `prefixItems` are used to navigate in the Json tree

In [8]:
list_json_test = [
 ['',                   '',         ''     , 2],
 ['/family',            'family',   'white', 1],
 ['/children',          'children', ''     , 4],
 ['/children/0',        '',         15     , 1],
 ['/children/1',        '',         21     , 1],
 ['/children/2',        '',         ''     , 1],
 ['/children/2/simon',  'walter jr',None   , 1],
 ['/children/3',        '',         ''     , 1],
 ['/children/3/judith', 'judith',   22     , 1] ]

list_json_schema = [
 ['/family',     {'maxLength':10}],
 ['/children',   {'maxItems': 4, 'items': {'maximum': 25, 'propertyNames': {'maxLength':10}}}] ]

# Example 3

In [9]:
ntv_test = {'family:string' : 'white', 
            'children': [{':int': 15}, 
                         21, 
                         {'walter jr':None}, 
                         {'judith': 22}]}

ntv_schema = {'properties': {
                  'family': {
                         'maxLength': 5},
                  'children':{  
                         'maxItems': 5,
                         'prefixItems':[
                             {'typeNtv': {'const':'int'}}],
                         'items': {
                             'maximum': 25,
                             'nameNtv': {'maxLength': 10}}
                   }}}
print(ntv_validate(ntv_test, ntv_schema, mode=1), '\n')

  validate :  
  validate :  /family:string
  validate :  /children
  validate :  /children/0
  validate :  /children/0
  validate :  /children/1
  validate :  /children/2
  validate :  /children/3
True 



In [10]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets::':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema3  = {
                'properties': {
                    'House.': {
                        'properties':{
                            'location':{
                                'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
                            'arrival': {
                                'typeNtv': {'enum': ['year', 'date', 'datetime']}}}},
                    'childrens':{  
                         ':maxItems': 5,
                         'items': {
                             'properties':{
                                 'age':{
                                     'maximum': 25,
                                     'typeNtv': {'enum': ['json', 'int']},
                                     }}}},
                    'pets':{
                       'prefixItems':[
                           {'typeNtv': {'const':'$dog'}}],
                       'items':{
                           'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},
                       'properties': {
                           ':$mouse': {'maximum': 10 }}
                       }
                    }
                }
ntv_schema4  = {       
                '/House.': {
                    '/location':{
                        'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
                    '/arrival': {
                        'typeNtv': {'enum': ['year', 'date', 'datetime']}}},
                '/childrens':{  
                     'maxItems': 5,
                     'items': {
                         '/age':{
                             'maximum': 25,
                             'typeNtv': {'enum': ['json', 'int']},
                             }}},
                '/pets':{
                   '/0':{ 
                        'typeNtv': {'const':'$dog'}},
                   'items':{
                       'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},                  
                    '/:$mouse': {'maximum': 10 }
                   }
                }
                    
print(ntv_validate(ntv_test2, ntv_schema3, mode=1))
print(ntv_validate(ntv_test2, ntv_schema4, mode=1))

  validate :  example family::org.
  validate :  example family::org./House.
  validate :  example family::org./House./location:point
  validate :  example family::org./House./arrival:year
  validate :  example family::org./childrens::Person.
  validate :  example family::org./childrens::Person./0
  validate :  example family::org./childrens::Person./0
  validate :  example family::org./childrens::Person./1
  validate :  example family::org./childrens::Person./2
  validate :  example family::org./childrens::Person./2/age
  validate :  example family::org./pets
  validate :  example family::org./pets/:$dog
  validate :  example family::org./pets/:$dog
  validate :  example family::org./pets/:$cat
  validate :  example family::org./pets/:$mouse
  validate :  example family::org./pets/:$mouse
True
  validate :  example family::org.
  validate :  example family::org./House.
  validate :  example family::org./House./location:point
  validate :  example family::org./House./arrival:year
  val

In [11]:
ntv_test2 = {
      'example family::org.' : {
             'name': 'white',
             'House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_test3 = {
      'example family' : {
             'name': 'white',
             'org.House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:org.Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema2  = {'nameNtv' : {
                       'maxLength': 50},
                'properties': {
                        'org.House.': {
                            'properties':{
                                'location':{
                                    'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
                                'arrival': {
                                    'typeNtv': {':enum': ['year', 'date', 'datetime']}}}},
                        'childrens':{  
                             'maxItems': 5,
                             'items': {
                                 'properties':{
                                     'age':{
                                         'maximum': 25,
                                         'typeNtv': {'enum': ['json', 'int']},
                                         }}}},
                        'pets':{
                           'prefixItems':[
                               {'typeNtv': {'const':'$dog'}}],
                           'items':{
                               'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},
                           'properties': {
                               ':$mouse': {'maximum': 10 }}
                           }
                       }}

ntv_schema3  = {'properties': {
                        'org.House.': {
                            'properties':{
                                'location':{
                                    'typeNtv': {'enum': ['point', 'geojson', 'org.Place.address']}},
                                'arrival': {
                                    'typeNtv': {'enum': ['year', 'date', 'datetime']}}}},
                        'childrens':{  
                             'maxItems': 5,
                             'items': {
                                 'properties':{
                                     'age':{
                                         'maximum': 25,
                                         'typeNtv': {':enum': ['json', 'int']},
                                         }}}},
                        'pets':{
                           'prefixItems':[
                               {'typeNtv': {'const':'$dog'}}],
                           'items':{
                               'typeNtv': {'enum': ['$dog', '$cat', '$mouse']}},
                           'properties': {
                               ':$mouse': {'maximum': 10 }}
                           }
                       }}
print(ntv_validate(ntv_test3, ntv_schema2, mode=3))

     example family/org.House./location:point - typeNtv  :  point {'enum': ['point', 'geojson', 'org.Place.address']}
     example family/org.House./arrival:year - typeNtv  :  year {'enum': ['year', 'date', 'datetime']}
     example family/childrens::org.Person./0 - valueNtv  :  21 {'maximum': 25}
     example family/childrens::org.Person./0 - typeNtv  :  json {'enum': ['json', 'int']}
     example family/childrens::org.Person./2/age - valueNtv  :  22 {'maximum': 25}
     example family/childrens::org.Person./2/age - typeNtv  :  json {'enum': ['json', 'int']}
     example family/childrens::org.Person. - valueNtv  :  [{"age": 21}, {":org.Person.givenName": "walter jr"}, {"org.Person.": {"age": 22, ":givenName": "judith"}}] {'maxItems': 5}
     example family/pets/:$dog - typeNtv  :  $dog {'const': '$dog'}
     example family/pets/:$dog - typeNtv  :  $dog {'enum': ['$dog', '$cat', '$mouse']}
     example family/pets/:$cat - typeNtv  :  $cat {'enum': ['$dog', '$cat', '$mouse']}
     examp

In [12]:
ntv_test3 = {
      'example family' : {
             'name:string': 'white',
             'home:org.House.':{
                 'location:point': [2.2, 48.5],
                 ':Country.name': 'France',
                 'arrival:year': 2005},
             'childrens:org.Person.': [ 
                          {'age': 21}, 
                          {':givenName': 'walter jr'}, 
                          {'age': 22, ':givenName': 'judith'}],
              'pets':{
                  ':$dog': {'hector': 10}, 
                  ':$cat': {'tom': 5},
                  ':$mouse': 3
               }}}

ntv_schema4  = {'sch.':{
#ntv_schema4  = {
                    'name' : {
                        ':maxLength': 50},
                    'home': {                        
                        ':items': {
                            ':nameNtv': {':maxLength': 50}},
                        'location':{
                            ':typeNtv': {':enum': ['point', 'geojson', 'org.Place.address']}},
                        'arrival': {
                            ':typeNtv': {':enum': ['year', 'date', 'datetime']}}},
                    'childrens':{  
                        ':maxItems': 5,
                        ':items': {
                            'age':{
                                ':maximum': 25,
                                ':typeNtv': {':enum': ['json', 'int']}}}},
                        'pets':{
                           '0':{
                               ':typeNtv': {':const':'$dog'}},
                           ':items':{
                               ':typeNtv': {':enum': ['$dog', '$cat', '$mouse']}},
                           ':$mouse': {
                               ':maximum': 10 }}
                    }
                }
print(ntv_validate2(ntv_test3, ntv_schema4, mode=2))

     example family/name:string - valueNtv  :  white {'maxLength': 50}
     example family/home::org.House./location:point - nameNtv  :  location {'maxLength': 50}
     example family/home::org.House./:Country.name - nameNtv  :   {'maxLength': 50}
     example family/home::org.House./arrival:year - nameNtv  :  arrival {'maxLength': 50}
     example family/home::org.House./location:point - typeNtv  :  point {'enum': ['point', 'geojson', 'org.Place.address']}
     example family/home::org.House./arrival:year - typeNtv  :  year {'enum': ['year', 'date', 'datetime']}
     example family/childrens::org.Person. - valueNtv  :  [{"age": 21}, {":org.Person.givenName": "walter jr"}, {"org.Person.": {"age": 22, ":givenName": "judith"}}] {'maxItems': 5}
     example family/childrens::org.Person./0 - valueNtv  :  21 {'maximum': 25}
     example family/childrens::org.Person./0 - typeNtv  :  json {'enum': ['json', 'int']}
     example family/childrens::org.Person./2/age - valueNtv  :  22 {'maximum': 

In [13]:
print(Ntv.obj(ntv_test2).name)
print(Ntv.obj(ntv_test2).pointer())
print(Ntv.obj({'org.':{':$dog': {'hector': 10}, ':$cat': {'tom': 5},':$mouse': 3}})[2].pointer())

example family
example family::org.
org./:$mouse


In [14]:
list_ntv_test = [
 ['',                   '',         ''     , None    , 2],
 ['/family',            'family',   'thomy', 'string', 1],
 ['/children',          'children', ''     , None    , 4],
 ['/children/0',        '',         15     , 'int'   , 1],
 ['/children/1',        '',         21     , 'json'  , 1],
 ['/children/simon',    'simon',    None   , 'json'  , 1],
 ['/children/judith',   'judith',   22     , 'json'  , 1] ]

list_ntv_schema = [
 ['/family',     None,          {':maxLength': 10 }], # ntv_value
 ['/children',   None,          {':maxItems': 4}], # ntv_value
 ['/children',   'items.',      {':maximum': 25}], # ntv_value
 ['/children',   'items.name.', {':maxLength': 10}] ] #ntv_name

list_ntv_schema2 = [
 ['/family',     None,          {':maxLength': 10 }  ],  # ntv_value
 ['/family',     'type.',       {':const': 'string'} ], # ntv_type
 ['/children',   None,          {':maxItems': 4}], # ntv_value
 ['/children',   'items.',      {':maximum': 25} ], # ntv_value
 ['/children',   'items.type.', {':enum': ['json', 'int']} ], # ntv_type
 ['/children',   'items.name.', {':maxLength': 10} ], # ntv_name
 ['/children/0', 'type.',       {':const':'int'}] ] # ntv_type

## JsonSchema keywords


### type
Type of a value 

* schema applicable to NTVvalue
```json
{ "type": ["number", "string"] } check if the NTVvalue is a number or a string
```

### String type
length, format, regex 

* schema applicable to NTVvalue 
```json
{ "type": "string", "minLength": 2, "maxLength": 3 } check the length of the NTVvalue
```


### Number type
multiples, range 

* schema applicable to NTVvalue
```json
{ "type": "number", "multipleOf" : 10 } check if the NTVvalue is a multiple of 10
```


### Object : properties
Schema for a value defined by a name 

* schema applicable to the NTVvalue of the NTV entity defined by his relative NTV-pointer 
```json
{ "properties": { "street_name": value_schema } } check the value_schema for the NTVvalue of the NTV entity defined by "street_name" (relative NTV-pointer)
 ```


### Object : patternProperties
Schema for values defined by a name matches a pattern 

* schema applicable to the NTVvalue of the NTV entity whose relative NTV-pointer matches a pattern.
```json
{ "patternProperties": { "^S_": pat_schema } } check the pat_schema for the NTVvalue of the NTV entity whose relative NTV-pointer matches "^S_"
 ```


### Object : additionalProperties
Schema for values whose names are not listed in the properties or patternProperties 

* schema applicable to the NTV entity not listed in the properties or patternProperties.
```json
{ "additionalProperties": add_schema } } check the add_schema for the NTVvalue of the NTV entity not listed in the properties or patternProperties.
 ```


### Object : propertyNames
Schema for names 

* schema applicable to the NTVname of the NTV entities 
```json
{ "propertyNames": names_schema } check the names_schema for the NTVnames of the NTV entity 
 ```


### Object : size
Number of properties 

* Number of NTV entities included in the NTV entity
```json
{ "minProperties": 2, "maxProperties": 3 } check the number of NTV entities 
 ```


### Object : other
Required : applicable,

unevaluatedProperties : idem as additionalProperties

### Array : items
Schema for values included in an array 

* schema applicable to the NTV entities included in the NTV entity 
```json
{ "items": items_schema } } check the items_schema for the NTVvalue of the NTV entities included in the NTV entity.
 ```


### Array : uniqueItems
Schema to ensure that each of the items in an array is unique 

* schema applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "uniqueItems": true } } check the uniqueness of the NTVvalue of the NTV entities.
 ```


### Array : prefixItems
Array of schemas for values included in an array 

* schemas applicable to the NTVvalue of the NTV entities included in the NTV entity 
```json
{ "prefixItems": [ item_schema ] } 
check the item_schema for the NTVvalue of the corresponding NTV entity included in the NTV entity.
 ```


### Array : contains, maxContains, minContains
schema for at least one (or number as defined with maxContains or minContains) value included in an array 

* schema applicable to the NTVvalues of the NTV entities included in the NTV entity 
```json
{ "contains": cont_schema, "minContains": 2, "maxContains": 3 } 
check if the cont_schema is valid for 2 or 3 NTVvalue of the NTV entity included.
 ```


### Array : other
unevaluatedItems : applies to any NTVvalues not evaluated by an items, prefixItems, or contains keyword.

minItems, maxItems : equivalent to minProperties, maxProperties


## NTV extension

### xxproperties
"pointer:part": schema

* pointer : relative NTV pointer (integer, NTVname, NTVtype)
* part : name, type or value (default)

### propertyNames 
equivalent to items:name

### required
list of pointers

### xxitems
"xxitems:part": schema

### xxcontains
"xxcontains:part": schema